In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
import os
import re
import sys
import cv2
import tqdm
import time
import spacy 
import random
import scipy.io
import itertools
import numpy as np
from math import ceil
import pandas as pd
from itertools import chain
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage import color
from scipy.ndimage.filters import gaussian_filter
from sklearn.model_selection import train_test_split
from skimage.transform import rescale, resize, downscale_local_mean

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence 
import torchvision.transforms as transforms
import json
import torch
import torchvision.transforms as T
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [ ]:
print('Downloading the COCO dataset ....')
!wget http://images.cocodataset.org/zips/train2017.zip

--2021-07-25 15:11:33--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... ^C


In [ ]:
!ls ./gdrive/MyDrive/Deep
!ls

ls: cannot access './gdrive/MyDrive/Deep': No such file or directory
sample_data


In [ ]:
print('Unzipping the downloaded dataset ...')
!unzip ./train2017.zip

Unzipping the downloaded dataset ...
unzip:  cannot find or open ./train2017.zip, ./train2017.zip.zip or ./train2017.zip.ZIP.


In [ ]:
class Data_prep():
    def __init__(self, DATA_PATH = './train2017/'):
        
        self.DATA_PATH = DATA_PATH
        print('Loading JSON file ....')
        f = open('./gdrive/MyDrive/Deep/instances_train2017.json')
        data = json.load(f)
        
        self.cats = {}
        for i in data['annotations']:
            if i['category_id'] in self.cats.keys():
                self.cats[i['category_id']].append(i['image_id'])
            else:
                self.cats[i['category_id']] = []
                self.cats[i['category_id']].append(i['image_id'])

        self.image_names = []
        self.refrence_names = []
        for k in [36, 42, 24, 35]:
            print('Loading class #' + str(k))
            class_data = []
            for i in data['images']:
                if i['id'] in self.cats[k]:
                    if not i['id'] in ['000000030349.jpg', '000000313608.jpg', '000000312288.jpg', '000000410498.jpg', '000000006379.jpg', '000000176483.jpg', '000000476888.jpg']:
                        class_data.append(i['file_name'])
            class_data_sorted = sorted(class_data)
            for i, j in zip(class_data, class_data_sorted):
                if not i in self.image_names:
                    self.image_names.append(i)
                    self.refrence_names.append(j)
        
        print('Loading images ...')

    def load_data(self):
        X_train, X_val, Y_train, Y_val = train_test_split(self.image_names, self.refrence_names, test_size=0.49, random_state=42)
        return X_train, X_val, Y_train, Y_val

In [ ]:
def load_image(path):
    try:
        DATA_PATH = './train2017/'
        image = resize(imread(DATA_PATH + path), (256, 256))
        image = np.transpose(image, (2,0,1))
    except:
        return []
    return image

def load_image_LAB(path):
    try:
        DATA_PATH = './train2017/'
        image = color.rgb2lab(resize(imread(DATA_PATH + path), (256, 256)))
        image = np.transpose(image, (2,0,1))
    except:
        return []
    return image

In [ ]:
class COCODataset(torch.utils.data.Dataset):
    def __init__(self, Image, Refrence, transform):
        self.Image = Image
        self.Refrence = Refrence
        self.transform = transform
    def __len__(self):
        return len(self.Image)
    def __getitem__(self, index):
        i = torch.tensor(load_image(self.Image[index]))
        r = torch.tensor(load_image(self.Refrence[index]))
        if len(i.shape) != 3 or len(r.shape) != 3:
            print('picture problem!!!!!!!!!!!!!!!!!!!!!!!!')
            i = torch.tensor(load_image('000000004704.jpg'))
            r = torch.tensor(load_image('000000332990.jpg'))
        i = self.transform(i)
        r = self.transform(r)
        return i, r

In [ ]:
class COCODataset_FullTrain(torch.utils.data.Dataset):
    def __init__(self, Image, Refrence, transform):
        self.Image = Image
        self.Refrence = Refrence
        self.transform = transform
    def __len__(self):
        return len(self.Image)
    def __getitem__(self, index):
        try:
            inputImg = torch.tensor(load_image_LAB(self.Image[index]))[0]
            refImg = torch.tensor(load_image_LAB(self.Refrence[index]))
            img_gt = torch.tensor(load_image_LAB(self.Image[index]))
        except:
            print('picture problem!')
            inputImg = torch.tensor(load_image_LAB('000000004704.jpg'))[0]
            refImg = torch.tensor(load_image('000000332990.jpg'))
            img_gt = torch.tensor(load_image_LAB('000000004704.jpg'))
        if len(img_gt.shape) != 3 or len(refImg.shape) != 3:
            print('picture problem!')
            inputImg = torch.tensor(load_image_LAB('000000004704.jpg'))[0]
            refImg = torch.tensor(load_image('000000332990.jpg'))
            img_gt = torch.tensor(load_image_LAB('000000004704.jpg'))

        inputImg = inputImg
        refImg = self.transform(refImg)
        img_gt = self.transform(img_gt)
        return inputImg, refImg, img_gt

In [ ]:
trans = T.Compose([
          T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])